In [ ]:
%load_ext autoreload
%autoreload 2
from lens_simulation.Lens import Lens, LensType, GratingSettings
from lens_simulation.Medium import Medium, LithiumNiabate

from lens_simulation.structures import SimulationOptions, SimulationParameters
from lens_simulation import utils
import matplotlib.pyplot as plt

import numpy as np


In [ ]:
# create lens
parameters = SimulationParameters(
    A=10000,
    pixel_size=1e-6,
    sim_height=1000e-6,
    sim_width=1000e-6,
    sim_wavelength=488e-9,
    lens_type=LensType.Spherical,
)



In [ ]:

from lens_simulation.Lens import test_escape_path_fits_inside_simulation
from lens_simulation.Simulation import pad_simulation

ep = 0.5
lens_type = LensType.Cylindrical
lens_type = LensType.Spherical

lens = Lens(500e-6, 50e-6, 2.0, LithiumNiabate())
lens.generate_profile(parameters.pixel_size, lens_type, length=200e-6)

print("lens: ", lens.profile.shape)
plt.imshow(lens.profile, cmap="plasma")
plt.title("lens profile")
plt.show()

# aperture
lens.create_custom_aperture(100e-6, 150e-6, type="radial", inverted=False)

# truncation
lens.create_truncation_mask(truncation_height=49e-6, aperture=True)

grating_settings = GratingSettings(
    width = 25e-6,
    distance = 50e-6,
    depth = 20e-6,
    centred = True
)

lens.create_grating_mask(grating_settings, True, True)

lens.apply_masks(grating=True, truncation=True)

# escape path
lens.create_escape_path(parameters, ep)

# pad simulation
lens = pad_simulation(lens, parameters)

# apply all mask
lens.apply_aperture_masks()

fig = utils.plot_apeture_masks(lens)
plt.show()

fig = utils.plot_lens_modifications(lens)
plt.show()



# TODO: add escape path to config


In [ ]:
test_escape_path_fits_inside_simulation(lens, parameters, ep=0.1)
test_escape_path_fits_inside_simulation(lens, parameters, ep=0.5)
test_escape_path_fits_inside_simulation(lens, parameters, ep=1.5)

In [ ]:
from lens_simulation import utils, validation
from lens_simulation.Lens import Lens, generate_lens, LensType
from lens_simulation.Medium import LithiumNiabate
from pprint import pprint

lens_config = utils.load_yaml_config("lens.yaml")

print("before validation")
pprint(lens_config)

lens_config = validation._validate_default_lens_config(lens_config)

print("after validation")
pprint(lens_config)

lens = generate_lens(lens_config, LithiumNiabate())

lens.generate_profile(1e-6)

# external pixel_size, lens_type

import matplotlib.pyplot as plt
plt.imshow(lens.profile)
plt.show()



#### Plot Slices as GIF
Plot the simulation slices as gif



In [ ]:
%load_ext autoreload
%autoreload 2
import os
from lens_simulation import utils
import glob
import imageio
import numpy as np
import matplotlib.pyplot as plt


print(os.getcwd())

path = r"\\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\rarely-upward-ocelot\safe-hound"

sim_path = os.path.join(path, str(0))
utils.save_propagation_gif(sim_path)

sim_path = os.path.join(path, str(1))
utils.save_propagation_gif(sim_path)


In [ ]:

# stack slices
sim_path = os.path.join(path, str(1))

utils.save_propagation_slices_gif(sim_path)

In [43]:

print(sim_path)

filenames = sorted(glob.glob(os.path.join(sim_path, "*mm.npy")))

# TODO: might not be possible for very large sims to load full sim, 
# will need to come up with another way to load slices in right format

# sim = None
# print(f"{len(filenames)} slices.")

# vslice = None
# hslice = None


# for i each dim in slice (301)
    # for each j slice (58)
        # get slice column
        # set to vslice arr column
        # vslice[j, i] = slice[i,]

# slice = np.load(filenames[0])
# vslice = np.zeros(shape=(len(filenames), slice.shape[0]))


# for i in range(slice.shape[0]):
#     for j, fname in enumerate(filenames):

#         slice = np.load(fname)
        
#         vslice[j, i] = slice[i, :] 

#         print(f"{i}, {j}")
#         print(f"fname: {os.path.basename(fname)}")
#         print(f"shape: {slice.shape}")
#         print(f"vslice: {vslice.shape})")
#         # print(f"hslice: {hslice.shape})")
    




\\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\rarely-upward-ocelot\safe-hound\1
58 slices.
0, 0
fname: 0.00000000mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 1
fname: 0.10325802mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 2
fname: 0.20651604mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 3
fname: 0.30977407mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 4
fname: 0.41303209mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 5
fname: 0.51629011mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 6
fname: 0.61954813mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 7
fname: 0.72280616mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 8
fname: 0.82606418mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 9
fname: 0.92932220mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 10
fname: 1.03258022mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 11
fname: 1.13583824mm.npy
shape: (301, 301)
vslice: (58, 301))
0, 12
fname: 1.23909627mm.npy
shape: (301, 301)
vslice: (58, 301))


KeyboardInterrupt: 

In [48]:
import os
import glob


directory = r"//ad.monash.edu/home/User028/pcle0002/Documents/repos/lens_simulation/src/lens_simulation/log/rarely-upward-ocelot"

sim_run_name = os.path.basename(directory)

sim_directories = [path for path in os.listdir(directory) if os.path.isdir(os.path.join(directory, path))] 

print("directory: ", directory)
print("sim_run_name: ", sim_run_name)
print("sim_dir: ", sim_directories)






directory:  //ad.monash.edu/home/User028/pcle0002/Documents/repos/lens_simulation/src/lens_simulation/log/rarely-upward-ocelot
sim_run_name:  rarely-upward-ocelot
sim_dir:  ['safe-hound']
['//ad.monash.edu/home/User028\\']
